In [1]:
import os
import json
from collections import defaultdict, Counter

# 1️⃣ Ensemble by Soft Voting

### 💡 soft voting ensemble code 구현 방법
1. 앙상블 할 `nbest_prediction.json` 파일을 하나의 폴더(`ex."./soft_voting"`) 에 저장
    - 이때 각 모델의 topk 값은 달라도 상관 없음
<br><br>

2. `prediction.json` id를 key 갖는 dictionaryf를 2개 생성
    - `dictionary 1` : key에 대한 value를 list로 생성. 이때 앙상블 할 nbest 파일에서 나오는 모든 ("text", "probability")쌍을 list의 개별 원소로 저장
    - `dictionary 2` : 최종 답안을 저장할 dictionary. 추후 json 파일로 저장됨
<br><br>

3. `dictionary 1`에서 중복되는 "text"에 대한 "probability"를 더하여 "text" 하나에 대해 하나의 "probability"만 갖도록 한 후,<br> 새롭게 생성된 "probability" 값 중 가장 큰 값에 해당하는 "text"만 저장하여 최종 답안 생성


In [12]:
file_path ='./soft_voting' # 앙상블 할 nbest_predictions.json 파일들이 저장된 폴더 경로

json_file_path = []
json_files=[]

for json_path in os.listdir(file_path):
    if json_path.endswith('.json'): # json file 만 가져오기
        json_file_path.append(json_path)

for file in json_file_path:
    fp=os.path.join(file_path,file)
    with open(fp,"r",encoding='utf-8') as json_file:
        json_data=json.load(json_file)
        json_files.append(json_data)

In [3]:
# 파일이름 리스트 출력
print(json_file_path)

['nbest_predictions(2).json', 'nbest_predictions(3).json', 'nbest_predictions(1).json']


In [4]:
key_list = list(json_files[0].keys() )# predictions에 들어가는 id 불러오기
candidates = defaultdict(list) # 모든 ("text", "probability") 쌍 저장할 dictionary (value : list 형태) 
answer_dict = defaultdict(list) # 최종 답변을 넣을 dict 생성 -> value : list

In [5]:
# json file에서 각 value의 가장 첫번째 값이 {} -> {}를 제거하는 코드
for json_file in json_files:
    for key in key_list:
        json_file[key] = json_file[key][1:]

In [6]:
for key in key_list:
    candidates[key]
    answer_dict[key]

for json_file in json_files:
    for key in key_list: # key example : "mrc-1-000653"
        for i in range(len(json_file[key])): # len(json_file[key]) : 각 실험에서 topk 설정 값
            candidates[key].append((json_file[key][i]['text'], json_file[key][i]['probability'])) # 모든 ("text", "probability")쌍 저장
            # 후처리 함수를 적용하여 사실상 같은 답안에 대해 더 높은 점수를 주는 것도 좋을 듯

In [7]:
for key in key_list:
    new_candidates = {} # text : probability
    for i in range(len(candidates[key])):                
        if candidates[key][i][0] not in new_candidates.keys():   # new_candidates에 없는 text 라면 새로 추가
            new_candidates[candidates[key][i][0]]=candidates[key][i][1]
        else:
            new_candidates[candidates[key][i][0]]+=candidates[key][i][1] # new_candidates에 없는 text 라면 기존 확률값에 더해주기 
    final_answer = max(new_candidates.keys(), key = lambda x: new_candidates[x]) # 최대 확률값에 해당하는 text를 최종 정답으로 저장
    answer_dict[key] = final_answer

In [8]:
new_nbest_json_path = "softvoted_prediction.json"

with open(new_nbest_json_path, "w", encoding="utf-8") as writer:
    writer.write(
    json.dumps(answer_dict, indent=4, ensure_ascii=False) + "\n"
    )

# 2️⃣ Ensemble by Hard Voting

### 💡hard voting ensemble code 구현 방법
1. 앙상블 할 `prediction.json` 파일을 하나의 폴더(`ex."./hard_voting"`) 에 저장
<br><br>

2. `prediction.json` id를 key 갖는 dictionaryf를 2개 생성
    - `dictionary 1` : key에 대한 value를 list로 생성. 이때 앙상블 할 prediction 파일에서 나오는 모든 정답을 리스트로 저장
    - `dictionary 2` : 최종 답안을 저장할 dictionary. 추후 json 파일로 저장됨
<br><br>

3. `dictionary 1`의 value 리스트에서 등장한 단어의 빈도수를 계산하여, 가장 많이 등장한 단어를 최종 답안으로<br> 다만 아직 most_common 아이템이 여러개일 경우 어떤 것을 정답으로 가져갈지는 구현이 잘 안되어있음(무조건 첫번째 값을 반환)


In [9]:
file_path ='./hard_voting' # 앙상블 할 predictions.json 파일들이 저장된 폴더 경로

json_file_path = []
json_files=[]

for json_path in os.listdir(file_path):
    if json_path.endswith('.json'): # json file 만 가져오기
        json_file_path.append(json_path)

for file in json_file_path:
    fp=os.path.join(file_path,file)
    with open(fp,"r",encoding='utf-8') as json_file:
        json_data=json.load(json_file)
        json_files.append(json_data)


In [10]:
candidates=defaultdict(list) 
answer_dict = defaultdict(list)

for key in key_list:
    candidates[key]
    answer_dict[key]

for json_file in json_files:
    for key in key_list:
        candidates[key].append(json_file[key])


In [ ]:
for key in candidates.keys():
    common = Counter(candidates[key]).most_common()
    answer_dict[key] = common[0][0] # most common이 1개가 아닐경우?

In [11]:
new_nbest_json_path = "hardvoted_prediction.json"

with open(new_nbest_json_path, "w", encoding="utf-8") as writer:
    writer.write(
    json.dumps(answer_dict, indent=4, ensure_ascii=False) + "\n"
    )